In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [8]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [28]:
pd.read_sql(
    """
    SELECT 'CRIM' as ColName, COUNT(*) as empty_cnt FROM boston
    where CRIM IS NULL
    UNION ALL
    SELECT 'ZN', COUNT(*) as cnt FROM boston
    where ZN IS NULL
    UNION ALL
    SELECT 'INDUS', COUNT(*) as cnt FROM boston
    where INDUS IS NULL
    UNION ALL
    SELECT 'CHAS', COUNT(*) as cnt FROM boston
    where CHAS IS NULL
    UNION ALL
    SELECT 'NOX', COUNT(*) as cnt FROM boston
    where NOX IS NULL
    limit 5
    
    """,
    con,
)

,ColName,empty_cnt
0,CRIM,0
1,ZN,0
2,INDUS,0
3,CHAS,0
4,NOX,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [31]:
pd.read_sql(
    """
    SELECT 'CRIM' as ColName, COUNT(DISTINCT CRIM) as unique_cnt FROM boston
    UNION ALL
    SELECT 'ZN', COUNT(DISTINCT ZN) as cnt FROM boston
    UNION ALL
    SELECT 'INDUS', COUNT(DISTINCT INDUS) as cnt FROM boston
    UNION ALL
    SELECT 'CHAS', COUNT(DISTINCT CHAS) as cnt FROM boston
    UNION ALL
    SELECT 'NOX', COUNT(DISTINCT NOX) as cnt FROM boston
    limit 5
    """,
    con,
)

,ColName,unique_cnt
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [94]:
pd.read_sql(
    """
    WITH add_rows as (
        SELECT 'ZN' col_name, ZN val, row_number() OVER(ORDER BY ZN DESC) rownum FROM boston
        UNION ALL
        SELECT 'INDUS' col_name, INDUS val, row_number() OVER(ORDER BY ZN DESC) rownum FROM boston
        UNION ALL
        SELECT 'CRIM' col_name, CRIM val, row_number() OVER(ORDER BY ZN DESC) rownum FROM boston
        UNION ALL
        SELECT 'CHAS' col_name, CHAS val, row_number() OVER(ORDER BY ZN DESC) rownum FROM boston
        UNION ALL
        SELECT 'NOX' col_name, NOX val, row_number() OVER(ORDER BY ZN DESC) rownum FROM boston
    ),
    med as (
        SELECT 'ZN' col_name, val from add_rows 
        WHERE rownum = (SELECT MAX(rownum)/2 med FROM add_rows) AND col_name = 'ZN'
        UNION ALL
        SELECT 'INDUS' col_name, val from add_rows 
        WHERE rownum = (SELECT MAX(rownum)/2 med FROM add_rows) AND col_name = 'INDUS'
        UNION ALL
        SELECT 'CRIM' col_name, val from add_rows 
        WHERE rownum = (SELECT MAX(rownum)/2 med FROM add_rows) AND col_name = 'CRIM'
        UNION ALL
        SELECT 'CHAS' col_name, val from add_rows 
        WHERE rownum = (SELECT MAX(rownum)/2 med FROM add_rows) AND col_name = 'CHAS'
        UNION ALL
        SELECT 'NOX' col_name, val from add_rows 
        WHERE rownum = (SELECT MAX(rownum)/2 med FROM add_rows) AND col_name = 'NOS'
    ),
    min as (
        SELECT 'ZN' col_name, MIN(ZN) min from boston
        UNION ALL
        SELECT 'INDUS' col_name, MIN(INDUS) min from boston
        UNION ALL
        SELECT 'CRIM' col_name, MIN(CRIM) min from boston
        UNION ALL
        SELECT 'CHAS' col_name, MIN(CHAS) min from boston
        UNION ALL
        SELECT 'NOX' col_name, MIN(NOX) min from boston
    )
    SELECT 'ZN' ColName from med , min
    WHERE (min.min = med.val AND min.col_name = 'ZN')
    UNION ALL
    SELECT 'CHAS' ColName from  med , min
    WHERE (min.min = med.val AND min.col_name = 'CHAS')
    UNION ALL
    SELECT 'CRIM' ColName from  med , min
    WHERE (min.min = med.val AND min.col_name = 'CRIM')
    UNION ALL
    SELECT 'INDUS' ColName from  med , min
    WHERE (min.min = med.val AND min.col_name = 'INDUS')
    UNION ALL
    SELECT 'NOX' ColName from  med , min
    WHERE (min.min = med.val AND min.col_name = 'NOX')
    """,
    con,
)

,ColName
0,ZN
1,CHAS


## Можно сделать вывод , что в колонках  ZN, CHAS не менее половины строк содержат одинаковые значения равные минимальному по колонке.

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [126]:
pd.read_sql(
    """
    WITH ordered_price as(
        SELECT RM, MEDV, COUNT() OVER() rows, row_number() OVER( ORDER BY MEDV DESC,RM DESC) rownum from boston    
    ),
    rm_max25 as (
        SELECT AVG(RM) val FROM ordered_price WHERE rownum <= 25
    ),
    rm_min25 as (
        SELECT AVG(RM) val FROM ordered_price WHERE rownum > rows - 25
    ),
    rm_max50 as (
        SELECT AVG(RM) val FROM ordered_price WHERE rownum <= 50
    ),
    rm_min50 as (
        SELECT AVG(RM) val FROM ordered_price WHERE rownum > rows - 50
    ),
    rm_max100 as (
        SELECT AVG(RM) val FROM ordered_price WHERE rownum <= 100
    ),
    rm_min100 as (
        SELECT AVG(RM) val FROM ordered_price WHERE rownum > rows - 100
    ),
    rm_max200 as (
        SELECT AVG(RM) val FROM ordered_price WHERE rownum <= 200
    ),
    rm_min200 as (
        SELECT AVG(RM) val FROM ordered_price WHERE rownum > rows - 200
    ),
    rm_max300 as (
        SELECT AVG(RM) val FROM ordered_price WHERE rownum <= 300
    ),
    rm_min300 as (
        SELECT AVG(RM) val FROM ordered_price WHERE rownum > rows - 300
    )
    SELECT '25' house_number, rm_max25.val max_rm, rm_min25.val  min_rm, rm_max25.val - rm_min25.val avg_price_diff FROM rm_max25, rm_min25
    UNION ALL
    SELECT '50' house_number, rm_max50.val  max_rm, rm_min50.val  min_rm,rm_max50.val - rm_min50.val FROM rm_max50, rm_min50
    UNION ALL
    SELECT '100' house_number, rm_max100.val  max_rm, rm_min100.val  min_rm,rm_max100.val - rm_min100.val FROM rm_max100, rm_min100
    UNION ALL
    SELECT '200' house_number, rm_max200.val  max_rm, rm_min200.val  min_rm,rm_max200.val - rm_min200.val FROM rm_max200, rm_min200
    UNION ALL
    SELECT '300' house_number, rm_max300.val  max_rm, rm_min300.val  min_rm,rm_max300.val - rm_min300.val FROM rm_max300, rm_min300


    """,
    con,
)

,house_number,max_rm,min_rm,avg_price_diff
0,25,7.637320,5.69412,1.943200
1,50,7.487060,5.75324,1.733820
2,100,7.198760,5.87511,1.323650
3,200,6.761530,5.91122,0.850310
4,300,6.539007,5.97121,0.567797


## Количество комнат существенно влияет на стоимость домов : все дома, в которых на 1 комнату больше стоят дороже .

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [134]:
pd.read_sql(
    """
    SELECT LSTAT, MEDV, RANK() OVER( ORDER BY LSTAT ASC) rank FROM boston
    ORDER BY MEDV DESC
    limit 20
    """,
    con,
)

,LSTAT,MEDV,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


## Достаточно большой расзброс по рангу LSTAT для домов с высокой ценой , это говорит о том, что дорогое жилье не сконцентрировано только в элитных районах а находится в районах , где проживают различные социальные группы.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [136]:
pd.read_sql(
    """
    SELECT DISTINCT CHAS, AVG(MEDV) OVER (PARTITION BY CHAS) avg_price FROM boston
    limit 5
    """,
    con,
)

,CHAS,avg_price
0,0.0,22.093843
1,1.0,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [143]:
pd.read_sql(
    """
    WITH yes_river as (
        SELECT 'CRIM' col , AVG(CRIM) val FROM boston WHERE CHAS = '1'
        UNION
        SELECT 'ZN', AVG(ZN) FROM boston WHERE CHAS = '1'
        UNION
        SELECT 'INDUS', AVG(INDUS) FROM boston WHERE CHAS = '1'
        UNION
        SELECT 'CHAS', AVG(CHAS) FROM boston WHERE CHAS = '1'
        UNION
        SELECT 'NOX', AVG(NOX) FROM boston WHERE CHAS = '1'
    ),
    no_river as (
        SELECT 'CRIM' col, AVG(CRIM) val FROM boston WHERE CHAS = '0'
        UNION
        SELECT 'ZN', AVG(ZN) FROM boston WHERE CHAS = '0'
        UNION
        SELECT 'INDUS', AVG(INDUS) FROM boston WHERE CHAS = '0'
        UNION
        SELECT 'CHAS', AVG(CHAS) FROM boston WHERE CHAS = '0'
        UNION
        SELECT 'NOX', AVG(NOX) FROM boston WHERE CHAS = '0'
    )
    SELECT 'CRIM' col_name FROM yes_river, no_river 
    where yes_river.val > no_river.val and yes_river.col = 'CRIM' and no_river.col = 'CRIM' 
    UNION
    SELECT 'ZN' FROM yes_river , no_river 
    where yes_river.val > no_river.val and yes_river.col = 'ZN' and no_river.col = 'ZN' 
    UNION
    SELECT 'INDUS' FROM yes_river , no_river 
    where yes_river.val > no_river.val and yes_river.col = 'INDUS' and no_river.col = 'INDUS' 
    UNION
    SELECT 'CHAS' FROM yes_river , no_river 
    where yes_river.val > no_river.val and yes_river.col = 'CHAS' and no_river.col = 'CHAS' 
    UNION
    SELECT 'NOX' FROM yes_river , no_river 
    where yes_river.val > no_river.val and yes_river.col = 'NOX' and no_river.col = 'NOX' 
    limit 5
    """,
    con,
)

,col_name
0,CHAS
1,INDUS
2,NOX


## концентрация NOX выше так как у реки местность в низине , и у рек располагается больше промышленных зон .

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [169]:
pd.read_sql(
    """
    with pin as (
        SELECT INDUS , PERCENT_RANK() OVER(ORDER BY INDUS ASC) as pin FROM boston
    ),
    pox as (
        SELECT NOX, PERCENT_RANK() OVER(ORDER BY NOX ASC) as pox FROM boston
    )
    SELECT 10 percintile, MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.1 and pox.pox <= 0.1
    UNION
    SELECT 20 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.2 and pox.pox <= 0.2
    UNION
    SELECT 30 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.3 and pox.pox <= 0.3
    UNION
    SELECT 40 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.4 and pox.pox <= 0.4
    UNION
    SELECT 50 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.5 and pox.pox <= 0.5
    UNION
    SELECT 60 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.6 and pox.pox <= 0.6
    UNION
    SELECT 70 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.7 and pox.pox <= 0.7
    UNION
    SELECT 80 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.8 and pox.pox <= 0.8
    UNION
    SELECT 90 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 0.9 and pox.pox <= 0.9
    UNION
    SELECT 100 , MAX(pin.INDUS) indus, MAX(pox.NOX) nox from pin, pox WHERE pin.pin <= 1 and pox.pox <= 1
    

    
    
    limit 100
    """,
    con,
)

,percintile,indus,nox
0,10,2.89,0.426
1,20,4.39,0.442
2,30,5.96,0.472
3,40,7.38,0.507
4,50,9.69,0.538
5,60,12.83,0.575
6,70,18.10,0.605
7,80,18.10,0.668
8,90,19.58,0.713
9,100,27.74,0.871


## Есть четкая взаимосвязь, что с ростом доли промышленной зоны растет и концентрация NOX